In [136]:
import pandas as pd
import numpy as np

In [137]:
ratings = pd.read_csv('movies/ratings.csv')
movies = pd.read_csv("movies/movies.csv")

In [138]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [139]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [140]:
ratings_df = ratings.pivot(
  index='movieId', 
  columns='userId')['rating'].fillna(0)
ratings_df

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
type(ratings_df)

pandas.core.frame.DataFrame

In [142]:
def get_rating(userid, movieid):
  try:
    return (ratings.loc[(ratings.userId==userid) & (ratings.movieId==movieid), 'rating'].iloc[0])
  except:
    return 0.0

In [143]:
get_rating(7,2)

0.0

### Hàm lấy vector của một user hoặc một item

In [144]:
def get_vector(type: str, id: int):
  if type == 'user':
    return ratings_df[id].values
  elif type == 'movie':
    return ratings_df.loc[id].values

In [145]:
test_user_vector = get_vector('user', 1)
print(test_user_vector)

[4. 0. 4. ... 0. 0. 0.]


In [146]:
type(test_user_vector)

numpy.ndarray

### Hai hàm tính độ tương đồng

In [147]:
def cosine_similarity(vector1, vector2):
  # Chỉ tính dựa trên các bộ phim mà cả 2 đã cùng đánh giá
  both_rated = 0
  vector1_filtered = []
  vector2_filtered = []
  
  for i in range(len(vector1)):
    if vector1[i] != 0 and vector2[i] != 0: # nếu ở vị trí i, cả 2 đều đã đánh giá
      vector1_filtered.append(vector1[i]) # giữ lại điểm đánh giá
      vector2_filtered.append(vector2[i]) 
      both_rated += 1

  if both_rated == 0:
    return 0
  
  else:
    vector1_filtered = np.array(vector1_filtered) # chuyển list thành numpy array
    vector2_filtered = np.array(vector2_filtered)

    # Tính cosine similarity
    # Tính tích vô hướng (dot product) của 2 vector
    dot_product = sum(a * b for a, b in zip(vector1_filtered, vector2_filtered))
    norm_vector1 = np.sqrt(sum(a ** 2 for a in vector1_filtered))
    norm_vector2 = np.sqrt(sum(b ** 2 for b in vector2_filtered))

    cosine_similarity = dot_product / (norm_vector1 * norm_vector2)

    return cosine_similarity
    

In [148]:
cosine_similarity(get_vector('user',  1), get_vector('user', 3))

np.float64(0.7919033104178548)

In [149]:
def pearson_similarity(vector1, vector2):
  # Chỉ tính dựa trên các bộ phim mà cả 2 đã cùng đánh giá
  both_rated = 0
  vector1_filtered = []
  vector2_filtered = []
  
  for i in range(len(vector1)):
    if vector1[i] != 0 and vector2[i] != 0: # nếu ở vị trí i, cả 2 đều đã đánh giá
      vector1_filtered.append(vector1[i]) # giữ lại điểm đánh giá
      vector2_filtered.append(vector2[i]) 
      both_rated += 1

  if both_rated == 0:
    return 0
  
  else:
    # Tính trung bình của 2 vector đã lọc
    n = len(vector1_filtered)

    sum_vector1 = 0
    sum_vector2 = 0
    for i in range(n):
        sum_vector1 += vector1_filtered[i]
        sum_vector2 += vector2_filtered[i]
    mean_vector1 = sum_vector1 / n
    mean_vector2 = sum_vector2 / n

     # Tính tử số và mẫu số
    numerator = 0
    sum_sq_v1 = 0
    sum_sq_v2 = 0

    for i in range(n):
        # Tử số
        diff_v1 = vector1_filtered[i] - mean_vector1
        diff_v2 = vector2_filtered[i] - mean_vector2
        numerator += diff_v1 * diff_v2

        # Mẫu số
        sum_sq_v1 += diff_v1 ** 2
        sum_sq_v2 += diff_v2 ** 2

    denominator = np.sqrt(sum_sq_v1) * np.sqrt(sum_sq_v2)

    # Tránh chia cho 0
    if denominator == 0:
        return 0

    return numerator / denominator

In [150]:
pearson_similarity(get_vector('user', 50), get_vector('user', 200))

np.float64(0.3960569995171616)

### Hàm lấy topk lân cận

In [151]:
def most_similar(target_vector: np.ndarray, ratings_df: pd.DataFrame, topk: int, similarity: str = 'cosine', axis: str = 'user'):
    if axis == 'user':
        ratings_df = ratings_df.T

    ids = ratings_df.index.tolist()
    scores = []

    for i, item_id in enumerate(ids):
        candidate_vector = ratings_df.iloc[i].values

        # Bỏ qua chính mình
        if np.array_equal(candidate_vector, target_vector):
            continue

        if similarity == 'cosine':
            sim = cosine_similarity(target_vector, candidate_vector)
        elif similarity == 'pearson':
            sim = pearson_similarity(target_vector, candidate_vector)
        
        scores.append((sim, item_id))

    scores.sort(key=lambda x: x[0], reverse=True)
    return scores[:topk]


In [152]:
most_similar(get_vector('user', 20), ratings_df, topk=20, similarity='pearson', axis='user')

[(np.float64(1.0000000000000002), 54),
 (np.float64(1.0000000000000002), 127),
 (np.float64(1.0000000000000002), 209),
 (np.float64(1.0000000000000002), 378),
 (np.float64(1.0000000000000002), 472),
 (np.float64(1.0), 56),
 (np.float64(1.0), 149),
 (np.float64(1.0), 337),
 (np.float64(1.0), 364),
 (np.float64(1.0), 442),
 (np.float64(1.0), 498),
 (np.float64(1.0), 511),
 (np.float64(0.9999999999999999), 296),
 (np.float64(0.9999999999999998), 157),
 (np.float64(0.9999999999999998), 175),
 (np.float64(0.9999999999999998), 183),
 (np.float64(0.9999999999999998), 358),
 (np.float64(0.9999999999999998), 386),
 (np.float64(0.9999999999999998), 444),
 (np.float64(0.9999999999999998), 516)]

In [153]:

# Danh sách các user đã rating cho bộ phim
def get_userids(movieid):
  return (ratings.loc[(ratings.movieId==movieid), 'userId'].tolist())

In [154]:
print(get_userids(50))

[1, 5, 6, 7, 8, 16, 17, 18, 23, 24, 28, 29, 32, 33, 35, 38, 39, 41, 42, 45, 46, 54, 57, 58, 60, 61, 63, 64, 66, 68, 69, 72, 80, 84, 88, 91, 96, 103, 105, 109, 112, 117, 122, 124, 131, 132, 133, 140, 142, 145, 152, 156, 160, 162, 166, 174, 177, 178, 182, 187, 191, 195, 197, 198, 199, 202, 211, 215, 217, 219, 220, 221, 222, 228, 233, 237, 239, 241, 246, 247, 249, 254, 260, 261, 262, 266, 268, 274, 275, 278, 279, 280, 282, 287, 290, 294, 295, 296, 297, 298, 301, 305, 307, 308, 309, 313, 314, 317, 322, 323, 326, 327, 328, 329, 330, 331, 332, 335, 336, 337, 338, 343, 346, 348, 352, 354, 356, 357, 362, 363, 368, 370, 372, 373, 380, 381, 382, 385, 386, 387, 391, 394, 396, 400, 411, 414, 415, 418, 419, 424, 425, 432, 433, 435, 437, 444, 446, 448, 449, 453, 455, 462, 469, 470, 472, 474, 480, 482, 483, 495, 497, 501, 510, 512, 515, 522, 524, 525, 541, 553, 555, 560, 561, 562, 565, 566, 567, 568, 569, 573, 580, 587, 588, 590, 593, 595, 596, 597, 599, 601, 602, 606, 608, 610]


In [155]:
import numpy as np
import pandas as pd

def get_recommendation(
    target_id: int,
    topN: int,
    topK: int,
    ratings_df: pd.DataFrame,
    similarity_name: str,
    mode: str # 'user' hoặc 'item'
):
    # Kiểm tra dữ liệu đầu vào
    if mode not in ['user', 'item']:
        raise ValueError("mode phải là 'user' hoặc 'item'")
    if mode == 'user' and target_id not in ratings_df.columns:
        raise ValueError(f"User ID {target_id} không tồn tại trong ratings_df")
    if mode == 'item' and target_id not in ratings_df.index:
        raise ValueError(f"Movie ID {target_id} không tồn tại trong ratings_df")

    # Xác định vector mục tiêu
    target_vector = ratings_df[target_id].values if mode == 'user' else ratings_df.loc[target_id].values

    # Tìm top K tương tự
    similar_entities = most_similar(
        target_vector=target_vector,
        ratings_df=ratings_df,
        topk=topK,
        similarity=similarity_name,
        axis=mode
    )

    # Lấy trung bình rating của entity cần dự đoán
    if mode == 'user':
        target_ratings = ratings_df[target_id]
    else:
        target_ratings = ratings_df.loc[target_id]

    rated_values = target_ratings[target_ratings > 0]
    mean_target = rated_values.mean() if not rated_values.empty else 0

    predictions, sum_similarity = {}, {}

    # Duyệt qua các neighbor
    for sim_score, neighbor_id in similar_entities:
        if neighbor_id == target_id:
            continue

        if mode == 'user':
            # User-based CF
            neighbor_ratings = ratings_df[neighbor_id]
            rated_movies = neighbor_ratings[neighbor_ratings > 0]
            mean_neighbor = rated_movies.mean() if not rated_movies.empty else 0

            for movieid in ratings_df.index:
                if ratings_df.loc[movieid, target_id] == 0:
                    rating_vi = neighbor_ratings[movieid]
                    if rating_vi > 0:
                        diff = rating_vi - mean_neighbor
                        predictions[movieid] = predictions.get(movieid, 0) + sim_score * diff
                        sum_similarity[movieid] = sum_similarity.get(movieid, 0) + sim_score

        else:
            # Item-based CF
            neighbor_ratings = ratings_df.loc[neighbor_id]
            rated_users = neighbor_ratings[neighbor_ratings > 0]
            mean_neighbor = rated_users.mean() if not rated_users.empty else 0

            for userid in ratings_df.columns:
                if ratings_df.loc[target_id, userid] == 0:
                    rating_uj = neighbor_ratings[userid]
                    if rating_uj > 0:
                        diff = rating_uj - mean_neighbor
                        predictions[userid] = predictions.get(userid, 0) + sim_score * diff
                        sum_similarity[userid] = sum_similarity.get(userid, 0) + abs(sim_score)

    # predicted rating
    ranking = []
    for key in predictions:
        if sum_similarity[key] > 0:
            predicted_rating = mean_target + (predictions[key] / sum_similarity[key])
            ranking.append((predicted_rating, key))

    ranking.sort(key=lambda x: x[0], reverse=True)
    return ranking[:topN]


## Yêu cầu 1: User-based Collaborative Filtering với độ tương đồng cosine

Xuất ra top 10 bộ phim (chưa được người dùng xem) có dự đoán rating cao nhất cho UserID = 20

**Thử nghiệm với K = 5 và K = 10**

In [156]:
# K = 5
recommendations_k5 = get_recommendation(target_id=20, topN=10, topK=5, ratings_df=ratings_df, similarity_name='cosine', mode='user')

for i, (pred_rating, movieid) in enumerate(recommendations_k5, 1):
    print(f"{i}. Movie ID: {movieid}| Predicted Rating: {pred_rating:.4f}")

1. Movie ID: 112| Predicted Rating: 5.2367
2. Movie ID: 260| Predicted Rating: 5.2367
3. Movie ID: 733| Predicted Rating: 5.2367
4. Movie ID: 737| Predicted Rating: 5.2367
5. Movie ID: 1210| Predicted Rating: 5.2367
6. Movie ID: 1429| Predicted Rating: 5.2367
7. Movie ID: 1476| Predicted Rating: 5.2367
8. Movie ID: 1639| Predicted Rating: 5.2367
9. Movie ID: 110| Predicted Rating: 5.1951
10. Movie ID: 282| Predicted Rating: 5.1951


In [157]:
# K = 10
recommendations_k10 = get_recommendation(target_id=20, topN=10, topK=10, ratings_df=ratings_df, similarity_name='cosine', mode='user')

for i, (pred_rating, movieid) in enumerate(recommendations_k10, 1):
    print(f"{i}. Movie ID: {movieid}| Predicted Rating: {pred_rating:.4f}")

1. Movie ID: 1036| Predicted Rating: 5.4270
2. Movie ID: 112| Predicted Rating: 5.2367
3. Movie ID: 733| Predicted Rating: 5.2367
4. Movie ID: 737| Predicted Rating: 5.2367
5. Movie ID: 1429| Predicted Rating: 5.2367
6. Movie ID: 1476| Predicted Rating: 5.2367
7. Movie ID: 1639| Predicted Rating: 5.2367
8. Movie ID: 110| Predicted Rating: 5.1951
9. Movie ID: 509| Predicted Rating: 5.1951
10. Movie ID: 337| Predicted Rating: 4.9590


## Yêu cầu 2: Item-based Collaborative Filtering với hệ số tương quan Pearson

Xuất ra 10 người sẽ rating cao nhất (chưa rating) cho bộ phim có movieId = 50

**Thử nghiệm với K = 5 và K = 10**

In [158]:
# K = 5
recommendations_movie_k5 = get_recommendation(target_id=50, topN=10, topK=5, ratings_df=ratings_df, similarity_name='pearson', mode='item')

for i, (pred_rating, userid) in enumerate(recommendations_movie_k5, 1):
    print(f"{i}. UserID: {userid:6d} | Predicted Rating: {pred_rating:.4f}")

1. UserID:    365 | Predicted Rating: 6.1377
2. UserID:    367 | Predicted Rating: 5.3627
3. UserID:     51 | Predicted Rating: 5.1377
4. UserID:    438 | Predicted Rating: 4.5377
5. UserID:    111 | Predicted Rating: 3.6377
6. UserID:    113 | Predicted Rating: 3.5377
7. UserID:    225 | Predicted Rating: 3.4044
8. UserID:    603 | Predicted Rating: 2.6377


In [159]:
# có lẽ chỉ cover được 8 users chưa rate vì ít neighbors 
# dẫn đến ít predictions (chỉ users mà các neighbors đó có rate)

In [160]:
# K = 10
recommendations_movie_k10 = get_recommendation(target_id=50, topN=10, topK=10, ratings_df=ratings_df, similarity_name='pearson', mode='item')

for i, (pred_rating, userid) in enumerate(recommendations_movie_k10, 1):
    print(f"{i}. UserID: {userid:6d} | Predicted Rating: {pred_rating:.4f}")

1. UserID:    365 | Predicted Rating: 6.1377
2. UserID:    367 | Predicted Rating: 5.3627
3. UserID:    452 | Predicted Rating: 5.3002
4. UserID:    600 | Predicted Rating: 4.9877
5. UserID:    288 | Predicted Rating: 4.8002
6. UserID:    514 | Predicted Rating: 4.7377
7. UserID:     51 | Predicted Rating: 4.5586
8. UserID:    438 | Predicted Rating: 4.5377
9. UserID:    318 | Predicted Rating: 4.4877
10. UserID:    572 | Predicted Rating: 3.9877


## Yêu cầu 3: So sánh và đánh giá kết quả giữa K = 5 và K = 10

In [161]:
# Yêu cầu 1: User-based CF cho User 20
k5_movies = [movieid for _, movieid in recommendations_k5]
k10_movies = [movieid for _, movieid in recommendations_k10]

# Tìm các movie giống nhau (intersection)
common_movies = set(k5_movies) & set(k10_movies)
num_common = len(common_movies)

# Tìm các movie khác nhau
diff_k5 = set(k5_movies) - set(k10_movies)
diff_k10 = set(k10_movies) - set(k5_movies)

print("Yêu cầu 1 (User-based, User 20):")
print(f"Số kết quả giống nhau: {num_common}/{min(len(k5_movies), len(k10_movies))}")
print(f"Kết quả giống: {sorted(common_movies)}")
print(f"Kết quả chỉ ở K=5: {sorted(diff_k5)}")
print(f"Kết quả chỉ ở K=10: {sorted(diff_k10)}")

# So sánh thứ tự của các kết quả giống
print("\nThứ tự của các kết quả giống:")
for movie in sorted(common_movies):
    pos_k5 = k5_movies.index(movie) + 1 if movie in k5_movies else "Không có"
    pos_k10 = k10_movies.index(movie) + 1 if movie in k10_movies else "Không có"
    print(f"Movie {movie}: Vị trí K=5: {pos_k5}, Vị trí K=10: {pos_k10}")

# Yêu cầu 2: Item-based CF cho Movie 50
k5_users = [userid for _, userid in recommendations_movie_k5]
k10_users = [userid for _, userid in recommendations_movie_k10]

# Tìm các user giống nhau (intersection)
common_users = set(k5_users) & set(k10_users)
num_common = len(common_users)

# Tìm các user khác nhau
diff_k5 = set(k5_users) - set(k10_users)
diff_k10 = set(k10_users) - set(k5_users)

print("\nYêu cầu 2 (Item-based, Movie 50):")
print(f"Số kết quả giống nhau: {num_common}/{min(len(k5_users), len(k10_users))}")
print(f"Kết quả giống: {sorted(common_users)}")
print(f"Kết quả chỉ ở K=5: {sorted(diff_k5)}")
print(f"Kết quả chỉ ở K=10: {sorted(diff_k10)}")

# So sánh thứ tự của các kết quả giống
print("\nThứ tự của các kết quả giống:")
for user in sorted(common_users):
    pos_k5 = k5_users.index(user) + 1 if user in k5_users else "Không có"
    pos_k10 = k10_users.index(user) + 1 if user in k10_users else "Không có"
    print(f"User {user}: Vị trí K=5: {pos_k5}, Vị trí K=10: {pos_k10}")

Yêu cầu 1 (User-based, User 20):
Số kết quả giống nhau: 7/10
Kết quả giống: [110, 112, 733, 737, 1429, 1476, 1639]
Kết quả chỉ ở K=5: [260, 282, 1210]
Kết quả chỉ ở K=10: [337, 509, 1036]

Thứ tự của các kết quả giống:
Movie 110: Vị trí K=5: 9, Vị trí K=10: 8
Movie 112: Vị trí K=5: 1, Vị trí K=10: 2
Movie 733: Vị trí K=5: 3, Vị trí K=10: 3
Movie 737: Vị trí K=5: 4, Vị trí K=10: 4
Movie 1429: Vị trí K=5: 6, Vị trí K=10: 5
Movie 1476: Vị trí K=5: 7, Vị trí K=10: 6
Movie 1639: Vị trí K=5: 8, Vị trí K=10: 7

Yêu cầu 2 (Item-based, Movie 50):
Số kết quả giống nhau: 4/8
Kết quả giống: [51, 365, 367, 438]
Kết quả chỉ ở K=5: [111, 113, 225, 603]
Kết quả chỉ ở K=10: [288, 318, 452, 514, 572, 600]

Thứ tự của các kết quả giống:
User 51: Vị trí K=5: 3, Vị trí K=10: 7
User 365: Vị trí K=5: 1, Vị trí K=10: 1
User 367: Vị trí K=5: 2, Vị trí K=10: 2
User 438: Vị trí K=5: 4, Vị trí K=10: 8


### Ảnh hưởng của tham số **K**

1. **K lớn (ví dụ K = 10)**

   * Mô hình xem xét nhiều láng giềng hơn, nhờ đó kết quả dự đoán thường **ổn định và đa dạng hơn**.
   * Ảnh hưởng của các **outlier** được giảm bớt do có sự “trung hòa” từ nhiều nguồn thông tin.
   * Tuy nhiên, nếu trong số đó có nhiều láng giềng **ít tương đồng thực sự**, độ chính xác có thể giảm xuống vì tín hiệu bị “loãng”.

2. **K nhỏ (ví dụ K = 5)**

   * Mô hình chỉ tập trung vào những láng giềng **gần gũi nhất và tương đồng cao nhất**, giúp dự đoán có thể **chính xác hơn trong các trường hợp dữ liệu rõ ràng**.
   * Nhược điểm là kết quả **dễ bị nhiễu** nếu trong nhóm nhỏ đó có outlier, đồng thời **thiếu tính đa dạng** trong khuyến nghị.
